In [1]:
%matplotlib inline
import matplotlib as mpl
plt = mpl.pyplot
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.autolimit_mode'] = 'round_numbers'
mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['axes.ymargin'] = 0
import numpy as np
import utils

In [2]:
# training_features, training_labels, validation_features, validation_labels = \
#     utils.get_training_data(0.2, onehot=False, standardize=False)
# print(training_features.shape, training_labels.shape,
#      validation_features.shape, validation_labels.shape)

In [3]:
# Tf-Idf transform
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf = TfidfTransformer()
# training_features = tfidf.fit_transform(training_features)
# validation_features = tfidf.transform(validation_features)

In [4]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Dropout

def create_model(dense=(160,), activation='sigmoid', dropout=0.7):
    model = Sequential()
    for d in dense:
        model.add(Dense(d, input_shape=(1000,)))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
class KerasClassifierMod(KerasClassifier):
    def fit(self, x, y, sample_weight, **kwargs):
        return super().fit(x, y, sample_weight=sample_weight, **kwargs)

Using TensorFlow backend.


In [5]:
nn = KerasClassifierMod(build_fn=create_model, epochs=1, batch_size=32, verbose=1)

In [6]:
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 50
learning_rate = 0.1
ada_nn = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)

In [7]:
# ada_nn.fit(training_features, training_labels)

In [8]:
from sklearn.metrics import zero_one_loss
def plot_ada_hist(ada, title):
    ada_acc = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(validation_features)):
        ada_acc[i] = zero_one_loss(pred_labels, validation_labels)

    ada_acc_train = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(training_features)):
        ada_acc_train[i] = zero_one_loss(pred_labels, training_labels)

    plt.figure(dpi=300)
    plt.plot(np.arange(n_estimators) + 1, ada_acc, label='AdaBoost Test Loss')
    plt.plot(np.arange(n_estimators) + 1, ada_acc_train, label='AdaBoost Train Loss')
    plt.semilogy()
    plt.legend()
    plt.title(title)
    plt.show()
    plt.clf()

In [9]:
# plot_ada_hist(ada_nn, 'AdaBoost with neural network\nfinal accuracy %.4f'
#              %ada_nn.score(validation_features, validation_labels))

In [10]:
# Now the final training
training_features1, training_labels1, _, _ = \
    utils.get_training_data(0, onehot=False, standardize=False)
print(training_features1.shape, training_labels1.shape)

(20000, 1000) (20000,)


In [11]:
test_features1 = utils.get_test_data(standardize=False)
print(test_features1.shape)

(10000, 1000)


In [12]:
# Tf-Idf transform
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
training_features1 = tfidf.fit_transform(training_features1)
test_features1 = tfidf.transform(test_features1)

In [13]:
nn = KerasClassifierMod(build_fn=create_model, epochs=3, batch_size=64, verbose=1)
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 1000
learning_rate = 0.02
ada_nn1 = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)
ada_nn1.fit(training_features1, training_labels1)

Epoch 1/3
20000/20000 [==============================] - 2s 110us/step - loss: 3.3313e-05 - acc: 0.6141
Epoch 2/3
20000/20000 [==============================] - 1s 69us/step - loss: 2.6809e-05 - acc: 0.7420
Epoch 3/3
20000/20000 [==============================] - 0s 24us/step
Epoch 1/3
20000/20000 [==============================] - 2s 118us/step - loss: 3.3038e-05 - acc: 0.6164
Epoch 2/3
20000/20000 [==============================] - 1s 69us/step - loss: 2.7039e-05 - acc: 0.7370
Epoch 3/3
20000/20000 [==============================] - 1s 25us/step
Epoch 1/3
20000/20000 [==============================] - 2s 115us/step - loss: 3.2745e-05 - acc: 0.6267
Epoch 2/3
20000/20000 [==============================] - 1s 69us/step - loss: 2.7017e-05 - acc: 0.7413
Epoch 3/3
20000/20000 [==============================] - 1s 27us/step
Epoch 1/3
20000/20000 [==============================] - 2s 119us/step - loss: 3.3908e-05 - acc: 0.6079
Epoch 2/3
20000/20000 [==============================] - 1s 69us/

20000/20000 [==============================] - 4s 180us/step - loss: 3.4530e-05 - acc: 0.6076
Epoch 2/3
20000/20000 [==============================] - 1s 75us/step - loss: 2.9174e-05 - acc: 0.7289
Epoch 3/3
20000/20000 [==============================] - 1s 56us/step
Epoch 1/3
20000/20000 [==============================] - 4s 182us/step - loss: 3.4651e-05 - acc: 0.6056
Epoch 2/3
20000/20000 [==============================] - 2s 76us/step - loss: 2.9286e-05 - acc: 0.7299
Epoch 3/3
20000/20000 [==============================] - 1s 59us/step
Epoch 1/3
20000/20000 [==============================] - 4s 186us/step - loss: 3.4800e-05 - acc: 0.6053
Epoch 2/3
20000/20000 [==============================] - 2s 76us/step - loss: 2.9590e-05 - acc: 0.7244
Epoch 3/3
20000/20000 [==============================] - 1s 58us/step
Epoch 1/3
20000/20000 [==============================] - 4s 192us/step - loss: 3.4744e-05 - acc: 0.5999
Epoch 2/3
20000/20000 [==============================] - 2s 78us/step - los

20000/20000 [==============================] - 5s 258us/step - loss: 3.5674e-05 - acc: 0.5985
Epoch 2/3
20000/20000 [==============================] - 2s 84us/step - loss: 3.1157e-05 - acc: 0.7153
Epoch 3/3
20000/20000 [==============================] - 2s 92us/step
Epoch 1/3
20000/20000 [==============================] - 5s 262us/step - loss: 3.5680e-05 - acc: 0.5936
Epoch 2/3
20000/20000 [==============================] - 2s 85us/step - loss: 3.1069e-05 - acc: 0.7198
Epoch 3/3
20000/20000 [==============================] - 2s 92us/step
Epoch 1/3
20000/20000 [==============================] - 5s 263us/step - loss: 3.6021e-05 - acc: 0.5827
Epoch 2/3
20000/20000 [==============================] - 2s 84us/step - loss: 3.1223e-05 - acc: 0.7177
Epoch 3/3
20000/20000 [==============================] - 2s 97us/step
Epoch 1/3
20000/20000 [==============================] - 5s 271us/step - loss: 3.5741e-05 - acc: 0.5949
Epoch 2/3
20000/20000 [==============================] - 2s 86us/step - los

20000/20000 [==============================] - 7s 338us/step - loss: 3.6579e-05 - acc: 0.5803
Epoch 2/3
20000/20000 [==============================] - 2s 92us/step - loss: 3.2387e-05 - acc: 0.7014
Epoch 3/3
20000/20000 [==============================] - 3s 131us/step
Epoch 1/3
20000/20000 [==============================] - 7s 346us/step - loss: 3.6993e-05 - acc: 0.5747
Epoch 2/3
20000/20000 [==============================] - 2s 94us/step - loss: 3.2442e-05 - acc: 0.7077
Epoch 3/3
20000/20000 [==============================] - 3s 132us/step
Epoch 1/3
20000/20000 [==============================] - 7s 341us/step - loss: 3.6285e-05 - acc: 0.5913
Epoch 2/3
20000/20000 [==============================] - 2s 92us/step - loss: 3.2385e-05 - acc: 0.7056
Epoch 3/3
20000/20000 [==============================] - 3s 132us/step
Epoch 1/3
20000/20000 [==============================] - 7s 346us/step - loss: 3.6659e-05 - acc: 0.5898
Epoch 2/3
20000/20000 [==============================] - 2s 93us/step - 

20000/20000 [==============================] - 9s 431us/step - loss: 3.7248e-05 - acc: 0.5822
Epoch 2/3
20000/20000 [==============================] - 2s 100us/step - loss: 3.3207e-05 - acc: 0.6952
Epoch 3/3
20000/20000 [==============================] - 3s 171us/step
Epoch 1/3
20000/20000 [==============================] - 9s 429us/step - loss: 3.7345e-05 - acc: 0.5726
Epoch 2/3
20000/20000 [==============================] - 2s 100us/step - loss: 3.3408e-05 - acc: 0.6883
Epoch 3/3
20000/20000 [==============================] - 3s 173us/step
Epoch 1/3
20000/20000 [==============================] - 9s 430us/step - loss: 3.7475e-05 - acc: 0.5700
Epoch 2/3
20000/20000 [==============================] - 2s 101us/step - loss: 3.3385e-05 - acc: 0.6892
Epoch 3/3
20000/20000 [==============================] - 4s 175us/step
Epoch 1/3
20000/20000 [==============================] - 9s 432us/step - loss: 3.7562e-05 - acc: 0.5691
Epoch 2/3
20000/20000 [==============================] - 2s 102us/ste

...

20000/20000 [==============================] - 6s 300us/step - loss: 3.7455e-05 - acc: 0.5154
Epoch 3/3
20000/20000 [==============================] - 48s 2ms/step
Epoch 1/3
20000/20000 [==============================] - 96s 5ms/step - loss: 3.9177e-05 - acc: 0.5094
Epoch 2/3
20000/20000 [==============================] - 6s 307us/step - loss: 3.7281e-05 - acc: 0.5276
Epoch 3/3
20000/20000 [==============================] - 48s 2ms/step
Epoch 1/3
20000/20000 [==============================] - 97s 5ms/step - loss: 3.9260e-05 - acc: 0.5000
Epoch 2/3
20000/20000 [==============================] - 6s 302us/step - loss: 3.7320e-05 - acc: 0.5268
Epoch 3/3
20000/20000 [==============================] - 48s 2ms/step
Epoch 1/3
20000/20000 [==============================] - 98s 5ms/step - loss: 3.9632e-05 - acc: 0.4973
Epoch 2/3
20000/20000 [==============================] - 6s 302us/step - loss: 3.7323e-05 - acc: 0.5171
Epoch 3/3
20000/20000 [==============================] - 48s 2ms/step
Epoch

20000/20000 [==============================] - 6s 310us/step - loss: 3.7372e-05 - acc: 0.5124
Epoch 3/3
20000/20000 [==============================] - 53s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 107s 5ms/step - loss: 3.9714e-05 - acc: 0.5100
Epoch 2/3
20000/20000 [==============================] - 6s 315us/step - loss: 3.6947e-05 - acc: 0.5353
Epoch 3/3
20000/20000 [==============================] - 54s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 107s 5ms/step - loss: 3.8772e-05 - acc: 0.5130
Epoch 2/3
20000/20000 [==============================] - 6s 313us/step - loss: 3.7382e-05 - acc: 0.5292
Epoch 3/3
20000/20000 [==============================] - 52s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 107s 5ms/step - loss: 3.9883e-05 - acc: 0.5019
Epoch 2/3
20000/20000 [==============================] - 6s 318us/step - loss: 3.7184e-05 - acc: 0.5346
Epoch 3/3
20000/20000 [==============================] - 53s 3ms/step
Ep

20000/20000 [==============================] - 6s 318us/step - loss: 3.7322e-05 - acc: 0.5126
Epoch 3/3
20000/20000 [==============================] - 56s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 115s 6ms/step - loss: 3.9756e-05 - acc: 0.5014
Epoch 2/3
20000/20000 [==============================] - 7s 325us/step - loss: 3.7467e-05 - acc: 0.5132
Epoch 3/3
20000/20000 [==============================] - 57s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 115s 6ms/step - loss: 3.9159e-05 - acc: 0.5189
Epoch 2/3
20000/20000 [==============================] - 7s 325us/step - loss: 3.7296e-05 - acc: 0.5281
Epoch 3/3
20000/20000 [==============================] - 57s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 117s 6ms/step - loss: 3.9314e-05 - acc: 0.5121
Epoch 2/3
20000/20000 [==============================] - 6s 320us/step - loss: 3.7239e-05 - acc: 0.5175
Epoch 3/3
20000/20000 [==============================] - 57s 3ms/step
Ep

20000/20000 [==============================] - 7s 329us/step - loss: 3.7179e-05 - acc: 0.5209
Epoch 3/3
20000/20000 [==============================] - 62s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 126s 6ms/step - loss: 3.9670e-05 - acc: 0.5000
Epoch 2/3
20000/20000 [==============================] - 7s 329us/step - loss: 3.7430e-05 - acc: 0.5145
Epoch 3/3
20000/20000 [==============================] - 62s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 126s 6ms/step - loss: 3.9274e-05 - acc: 0.5136
Epoch 2/3
20000/20000 [==============================] - 7s 335us/step - loss: 3.7034e-05 - acc: 0.5389
Epoch 3/3
20000/20000 [==============================] - 63s 3ms/step
Epoch 1/3
20000/20000 [==============================] - 126s 6ms/step - loss: 3.9101e-05 - acc: 0.4990
Epoch 2/3
20000/20000 [==============================] - 7s 331us/step - loss: 3.7427e-05 - acc: 0.5040
Epoch 3/3
20000/20000 [==============================] - 63s 3ms/step
Ep

KeyboardInterrupt: 

In [15]:
test_labels1 = ada_nn1.predict(test_features1)
utils.save_prediction(test_labels1, 'data/test_labels_ada_nn_2.txt')

10000/10000 [==============================] - 3s 291us/step


10000/10000 [==============================] - 3s 285us/step


10000/10000 [==============================] - 3s 284us/step


10000/10000 [==============================] - 3s 286us/step


10000/10000 [==============================] - 9s 887us/step


In [16]:
print(ada_nn1.score(training_features1, training_labels1))
# The training accuracy
# For 200 estimators and a learning rate of 0.05 it was 0.86925
# For ANN with 2 batches it was 0.8694

 5440/20000 [=======>......................] - ETA: 4s

KeyboardInterrupt: 